In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### The above code shows where the files are stored.

In [2]:
# Use pandas read_csv to load csv into notebook
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

# prints the first 5 rows of the data
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Use pandas read_csv to load csv into notebook
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

# prints the first 5 rows of the data
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preparing the Dataset

Let's create functions to handle some missing data.
After perusing the tables, we see there are missing data in the "Age", "Cabin", "Embarked" and "Fare" columns.

In [4]:
# define functions to fill missing values with mode or mean
def nan_to_mode(df_col):
    mode = df_col.mode()
    df_col.fillna(mode, inplace = True)
    return df_col

def nan_to_mean(df_col):
    mean = df_col.mean()
    df_col.fillna(mean, inplace = True)
    return df_col

In [5]:
# replace missing values with the mode or average values in each set
nan_to_mode(train_data.Embarked)
nan_to_mode(test_data.Embarked)

nan_to_mean(train_data.Fare)
nan_to_mean(test_data.Fare)

nan_to_mean(train_data.Age)
nan_to_mean(test_data.Age)

print("Missing values replaced successfully!")

Missing values replaced successfully!


Let's pay close attention to the "Cabin" column. Most of the data is missing. 

In [6]:
cabin_data = train_data["Cabin"].isna()
train_data['Missing Cabin #'] = cabin_data
train_data[["Survived","Missing Cabin #"]].groupby(['Survived'],as_index = False).value_counts()

,Survived,Missing Cabin #,count
0,0,True,481
1,0,False,68
2,1,True,206
3,1,False,136


### We see that the majority of survivors have a cabin number unlike ones who did not survive.
This means that if there is no cabin number, the chances that a person survived are low.
Lets make this into a binary feature.

In [7]:
train_data.Cabin.fillna(0, inplace = True)
for x in range(len(train_data.Cabin)):
    if train_data.Cabin[x] != 0:
        train_data.Cabin[x] = 1

test_data.Cabin.fillna(0, inplace = True)
for x in range(len(test_data.Cabin)):
    if test_data.Cabin[x] != 0:
        test_data.Cabin[x] = 1
        
print("Missing values replaced successfully!")

Missing values replaced successfully!


/tmp/ipykernel_20/2469313052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.Cabin[x] = 1
/tmp/ipykernel_20/2469313052.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.Cabin[x] = 1


## Training the model

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch","Embarked","Cabin","Age"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# Normalizing the values of the training and test set
X = (X - X.mean())/X.std()
X_test = (X_test - X_test.mean())/X_test.std()

model = RandomForestClassifier(n_estimators=100, max_depth = 4, random_state = 0)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index = False)
print("Your submission was successfully saved!")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_20/1362932633.py:7: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X = pd.get_dummies(train_data[features])
/tmp/ipykernel_20/1362932633.py:8: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X_test = pd.get_dummies(test_data[features])


Your submission was successfully saved!


In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0).fit(X, y)
print("Training score: ", f1_score(y, model.predict(X)))

Training score:  0.7338345864661654


In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
